<a href="https://colab.research.google.com/github/lhuissier4/Atelier_entrepot_de_donnee_epsi/blob/main/Atelier_entrepot_de_donnees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>